In [1]:
from utils.geopandas_to_folium import plot
import psycopg2
import folium
import gpxpy
from stmatching.db_utils import OSMDB
from stmatching.map_matcher import MapMatcher
import geopandas as gpd
from shapely.wkt import loads

In [2]:
conn = psycopg2.connect("dbname='melbourne_osm_test'")
db = OSMDB('ways',conn)
matchers = []
with open('./4_2_Route.gpx','r') as gpx_file:
	gpx = gpxpy.parse(gpx_file)
	for track in gpx.tracks:
		for segment in track.segments:
			pts = []
			for point in segment.points:
				pts.append({'coords':(point.longitude,point.latitude),
							'timestamp':point.time.timestamp(),
                            'speed':point.speed})
			pts = pts[100:300]
			matcher = MapMatcher(db,pts,30,6,10)
			matchers.append(matcher)
matcher = matchers[0]
route_edges = [str(x[1]) for x in matcher.get_path()]
route_edges = route_edges[:-1]
sql_str = '''select gid, the_geom from ways where gid in ({id_list})'''
df_edges = gpd.GeoDataFrame.from_postgis(sql_str.format(id_list=','.join(route_edges)),conn,geom_col="the_geom")

In [3]:
fmap = folium.Map(location=[-37.81,144.96],zoom_start=12)
plot(df_edges,fmap)

In [4]:
# add original points
for level in matcher.weighting_network:
    coords = level['observation']['coords']
    marker = folium.CircleMarker([coords[1],coords[0]],radius=4,color='red')
    marker.add_to(fmap)

# add matched points
for snaped_point in matcher.get_matched():
    pt = loads(snaped_point['geom_wkt'])
    marker = folium.CircleMarker([pt.y,pt.x],radius=4,color = 'green')
    marker.add_to(fmap)
    
fmap

In [5]:
import gpxpy.gpx
from datetime import datetime
gpx = gpxpy.gpx.GPX()
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)
for point in matcher.get_matched():
    pt = loads(point['geom_wkt'])
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(pt.y,pt.x,
                                                      speed=point['observation']['speed'],
                                                      time=datetime.fromtimestamp(point['observation']['timestamp'])))
print(gpx.to_xml())

<?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.topografix.com/GPX/1/0" xsi:schemaLocation="http://www.topografix.com/GPX/1/0 http://www.topografix.com/GPX/1/0/gpx.xsd" version="1.0" creator="gpx.py -- https://github.com/tkrajina/gpxpy">
<trk>
<trkseg>
<trkpt lat="-37.910735150222" lon="145.040281655543">
<time>2017-04-19T09:36:49Z</time>
<speed>5.722</speed></trkpt>
<trkpt lat="-37.9106919178487" lon="145.040288766682">
<time>2017-04-19T09:36:50Z</time>
<speed>5.278</speed></trkpt>
<trkpt lat="-37.9106451187964" lon="145.040296464491">
<time>2017-04-19T09:36:51Z</time>
<speed>5.278</speed></trkpt>
<trkpt lat="-37.9105929506204" lon="145.040305045448">
<time>2017-04-19T09:36:52Z</time>
<speed>5.278</speed></trkpt>
<trkpt lat="-37.9105542766032" lon="145.040311406798">
<time>2017-04-19T09:36:53Z</time>
<speed>5.306</speed></trkpt>
<trkpt lat="-37.9105155865707" lon="145.040317770784">
<time>2017-04-19T09:36:54Z</time>
<

In [6]:
with open('./4_2_Route.result2.gpx','w') as file:
    file.write(gpx.to_xml())